In [9]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock
import talib as ta

RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70

In [6]:
 def calculate_first_mondays(dates):
        if not isinstance(dates, pd.DatetimeIndex):
            dates = pd.DatetimeIndex(dates)
        dates_series = pd.Series(dates, index=dates)
        mondays = dates_series[dates_series.dt.dayofweek == 0]
        first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
        return set(first_mondays)

In [8]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 50  # Initialize the investment fund

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, timeperiod=14)
        # Calculate Bollinger Bands
        self.boll_high = self.I(ta.BBANDS, close, timeperiod=15, nbdevup=2, nbdevdn=2)[0]
        self.boll_low = self.I(ta.BBANDS, close, timeperiod=15, nbdevup=2, nbdevdn=2)[2]
        self.first_mondays = calculate_first_mondays(self.data.index)
    def next(self):
        today = self.data.index[-1]
        if today in self.first_mondays:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage

        # Check for buy signal
        if (self.data.Close[-1] <= self.boll_low[-1] and self.rsi[-1] <= 30):
            share_price = self.data.Close[-1]
            shares_to_buy = self.fund // share_price
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                
def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    bt.plot(filename=f'{stock_symbol}')
    
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['VCB', 'VPB', 'FPT', 'REE'] 

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-10 18:11:50,444 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; u

Results for VCB:
Total investment: 2548.5693986009032
Current Shares: 973
Current Equity: 3380.0
RoR: 32.62342402194462
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Results for VPB:
Total investment: 2349.671506331387
Current Shares: 4674
Current Equity: 3485.564645426516
RoR: 48.34263581246868
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Results for FPT:
Total investment: 2348.4988569089387
Current Shares: 1480
Current Equity: 5059.68345323741
RoR: 115.44330065788893
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Results for REE:
Total investment: 2898.650632457136
Current Shares: 2043
Current Equity: 4139.75210688592
RoR: 42.816525059341956
--------------------------------------------------
